In [1]:
# Import libraries
import numpy as np
import pandas as pd
# import pymysql
import psycopg2

# from sklearn import (metrics, linear_model)
# import xgboost as xgb
# from sklearn.model_selection import train_test_split

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
dbname = 'pic'
schema = 'pic'

In [3]:
# Connect to PIC
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password='password')
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

In [4]:
#patient characteristics
query = \
"""
WITH patientcharac as (
SELECT a.subject_id, a.hadm_id, i.icustay_id, 
    a.admittime as hosp_admittime, a.dischtime as hosp_dischtime, 
    a.hospital_expire_flag,
    i.first_careunit, i.los,
    p.dob, p.gender, 
    EXTRACT(EPOCH FROM (a.admittime - p.dob)/60/60/24/365) AS age_hosp_in
FROM icustays i
INNER JOIN admissions a
ON a.hadm_id = i.hadm_id
INNER JOIN patients p
ON a.subject_id = p.subject_id
)
SELECT * from patientcharac
where age_hosp_in BETWEEN 3 AND 5 
    AND hospital_expire_flag = 0;
"""
# select patients between 3-5 and exclude deaths

query_output = pd.read_sql_query(query,con)
query_output

C:\Users\feiya\AppData\Local\Temp\ipykernel_2112\2620579429.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query_output = pd.read_sql_query(query,con)


,subject_id,hadm_id,icustay_id,hosp_admittime,hosp_dischtime,hospital_expire_flag,first_careunit,los,dob,gender,age_hosp_in
0,45,100017,200549,2060-12-18 13:22:43,2060-12-28 14:27:00,0,General ICU,10.0451,2057-11-17 00:00:00,F,3.089198
1,53,100025,200557,2082-01-28 15:12:06,2082-01-30 23:41:00,0,General ICU,2.3535,2078-05-25 00:00:00,M,3.683927
2,68,100038,200570,2096-01-15 14:27:11,2096-01-19 14:04:00,0,General ICU,3.9840,2092-06-30 00:00:00,M,3.546855
3,9,100047,200579,2108-12-27 14:43:25,2108-12-28 00:32:00,0,General ICU,0.4090,2104-03-09 00:00:00,M,4.807160
4,89,100060,200592,2093-02-28 09:45:12,2093-03-10 11:55:00,0,General ICU,10.0903,2089-04-10 00:00:00,M,3.891524
...,...,...,...,...,...,...,...,...,...,...,...
1106,13526,114076,214418,2097-10-11 03:48:03,2097-10-12 08:30:00,0,SICU,0.9472,2092-10-14 00:00:00,F,4.994954
1107,13529,114079,214421,2070-11-07 09:23:32,2070-11-18 11:51:00,0,CICU,1.8965,2065-12-12 00:00:00,F,4.907921
1108,13555,114104,214442,2078-11-29 10:21:11,2078-12-05 11:17:00,0,PICU,1.7722,2074-08-04 10:20:00,M,4.323290
1109,13618,114174,214494,2075-12-19 10:05:57,2075-12-23 11:22:00,0,SICU,0.2194,2071-09-23 12:03:00,F,4.240873


In [5]:
# delete patients who had multiple ICU stay during a hospital stay
query1 = \
"""
WITH multiplestay AS (
SELECT hadm_id, count(hadm_id) as obs
FROM icustays
GROUP BY hadm_id)
SELECT *
FROM multiplestay
where obs>1;
"""
multiplestay = pd.read_sql_query(query1,con)
query_output=query_output[~query_output.hadm_id.isin(multiplestay.hadm_id)]

C:\Users\feiya\AppData\Local\Temp\ipykernel_2112\1221450027.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  multiplestay = pd.read_sql_query(query1,con)


In [6]:
#vital_signs
query2 = \
"""
SELECT hadm_id, itemid, charttime, storetime, valuenum, valueuom
FROM chartevents;
"""

vital_sign = pd.read_sql_query(query2,con)
vital_sign=vital_sign[vital_sign['itemid'].isin(['1012'])]
query_output=query_output[query_output.hadm_id.isin(vital_sign['hadm_id'])]

C:\Users\feiya\AppData\Local\Temp\ipykernel_2112\1868049271.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vital_sign = pd.read_sql_query(query2,con)


In [7]:
print(query_output.shape[0])
query_output.head()

358


,subject_id,hadm_id,icustay_id,hosp_admittime,hosp_dischtime,hospital_expire_flag,first_careunit,los,dob,gender,age_hosp_in
92,1330,101320,201851,2066-12-20 13:39:25,2067-02-01 16:00:00,0,General ICU,43.0979,2062-08-08 11:00:00,F,4.370166
266,3222,103295,203760,2097-03-18 10:53:48,2097-03-20 12:05:00,0,PICU,2.0500,2093-03-11 00:00:00,F,4.023162
269,3265,103337,203802,2093-04-27 12:36:59,2093-05-19 09:49:00,0,PICU,21.8840,2088-10-19 00:00:00,M,4.524728
274,3377,103447,203912,2086-07-22 11:25:20,2086-07-29 13:04:00,0,PICU,7.0688,2083-05-25 00:00:00,M,3.162948
299,4031,104087,204553,2101-01-18 08:44:22,2101-01-28 13:37:00,0,PICU,10.2035,2097-09-12 00:00:00,M,3.351683


In [9]:
data=vital_sign[vital_sign['hadm_id']==103337]
data[data['itemid']=='1012'].sort_values(by = ['charttime'])

,hadm_id,itemid,charttime,storetime,valuenum,valueuom
460937,103337,1012,2093-04-27 13:33:43,2093-04-27 13:33:43,0.0,None
466095,103337,1012,2093-06-08 13:01:19,2093-06-08 13:01:19,0.0,None


In [91]:
def dataprocess(x):
    data=vital_sign[vital_sign['hadm_id']==x]
    temperature_mean=data[data['itemid']=='1001'].mean()
    temperature_std=data[data['itemid']=='1001'].std()
    pulse_mean=data[data['itemid']=='1002'].mean()
    pulse_std=data[data['itemid']=='1002'].std()
    heartrate_mean=data[data['itemid']=='1003'].mean()
    heartrate_std=data[data['itemid']=='1003'].std()
    SBP_mean=data[data['itemid']=='1004'].mean()
    SBP_std=data[data['itemid']=='1004'].std()
    DBP_mean=data[data['itemid']=='1012'].mean()
    DBP_std=data[data['itemid']=='1012'].std()
    return temperature_mean,temperature_std,pulse_mean,pulse_std,heartrate_mean,heartrate_std,SBP_mean,SBP_std,DBP_mean,DBP_std
query_output['vital_sign']=query_output['hadm_id'].apply(dataprocess)
query_output.head()

C:\Users\feiya\AppData\Local\Temp\ipykernel_19208\453853135.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  temperature_mean=data[data['itemid']=='1001'].mean()
C:\Users\feiya\AppData\Local\Temp\ipykernel_19208\453853135.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  temperature_mean=data[data['itemid']=='1001'].mean()
C:\Users\feiya\AppData\Local\Temp\ipykernel_19208\453853135.py:4: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warnin

,subject_id,hadm_id,icustay_id,hosp_admittime,hosp_dischtime,hospital_expire_flag,first_careunit,los,dob,gender,age_hosp_in,vital_sign
1221,556,101222,201753,2082-06-03 09:23:33,2082-07-09 22:27:00,1,General ICU,36.5444,2075-05-02,F,7.094223,"([101222.0, 1001.0, 33.0], [nan, NaT, NaT, nan..."
1246,1254,101247,201778,2108-11-13 11:58:55,2108-11-16 15:01:00,0,General ICU,3.1271,2108-03-06,F,0.691779,"([101247.0, 33370003667.0, 36.0], [0.0, 1 days..."
1247,1241,101248,201779,2102-11-21 16:05:21,2102-12-03 12:55:00,1,General ICU,11.8681,2102-07-19,M,0.344302,"([101248.0, 9.100910091009101e+41, 36.65454545..."
1248,1255,101249,201780,2079-10-25 13:32:30,2079-11-10 16:17:00,1,General ICU,16.1146,2078-11-30,F,0.902916,"([101249.0, 1.112333455567779e+34, 36.41111111..."
1252,1260,101253,201784,2092-09-04 09:42:19,2092-09-14 11:48:00,0,General ICU,10.0875,2091-02-04,F,1.584670,"([101253.0, 1.2513751375137514e+30, 37.3375], ..."


In [99]:
query_output['vital_sign']

KeyError: 'vital_sign'

In [12]:
patient_characteristics['temperature_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[0]
patient_characteristics['temperature_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[1]
patient_characteristics['pulse_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[2]
patient_characteristics['pulse_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[3]
patient_characteristics['heartrate_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[4]
patient_characteristics['heartrate_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[5]
patient_characteristics['SBP_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[6]
patient_characteristics['SBP_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[7]
patient_characteristics['DBP_mean']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[8]
patient_characteristics['DBP_std']=patient_characteristics['vital_sign'].astype(str).str.split(',').str[9]
patient_characteristics['temperature_mean']=patient_characteristics['temperature_mean'].apply(lambda x:x[1:])
patient_characteristics['DBP_std']=patient_characteristics['DBP_std'].apply(lambda x:x[:len(x)-1])
def processstring(x):
    if x==' nan':
        x=np.nan
    else:
        x=round(float(x),2)
    return x
for c in ['temperature_mean','temperature_std','pulse_mean','pulse_std','heartrate_mean'
          ,'heartrate_std','SBP_mean','SBP_std','DBP_mean','DBP_std']:
    patient_characteristics[c]=patient_characteristics[c].apply(processstring)
def processgender(x):
    if x=='F':
        x='1'
    else:
        x='0'
    x=int(x)
    return x
patient_characteristics['gender']=patient_characteristics['gender'].apply(processgender)

In [15]:
#xgboost prediction model
X=patient_characteristics[['gender','age_hosp_in','temperature_mean','temperature_std','pulse_mean','pulse_std','heartrate_mean'
          ,'heartrate_std','SBP_mean','SBP_std','DBP_mean','DBP_std']]
y=patient_characteristics['icu_los']
a_train, a_test, b_train, b_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf_xgb=xgb.XGBRegressor(booster='gbtree',objective="reg:linear")
clf_xgb.fit(a_train, b_train)
prediction_xgb=clf_xgb.predict(a_test)
print('\nMean_absolute_error: {}'\
    .format( metrics.mean_absolute_error(b_test, prediction_xgb)))
print('\nExplained_variance_score: {}'\
    .format( metrics.explained_variance_score(b_test, prediction_xgb)))


Mean_absolute_error: 6.376199898192746

Explained_variance_score: 0.39886204568062733
